In [34]:
import sys
sys.path.append('/home/kelvin.wong/Developer/chexpert-fusion/fusion_experiments')

import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [54]:
from config.chexpert import PAPER_TRAINING_CLASSES

DATASET_PATH = '/home/kelvin.wong/Datasets/CheXpert-v1.0/train/'

FRONTAL_EXPERIMENT_PATH = '/home/kelvin.wong/experiments/chexpert_baseline_frontal_test_kelvin/'
FRONTAL_PREDICTIONS_PATH = os.path.join(FRONTAL_EXPERIMENT_PATH, 'predictions')

LATERAL_EXPERIMENT_PATH = '/home/kelvin.wong/experiments/chexpert_baseline_lateral_test_kelvin/'
LATERAL_PREDICTIONS_PATH = os.path.join(EXPERIMENT_PATH, 'predictions')

def compute_class_to_sorted_studies(predictions_path, classes):
    study_to_deltas = {}
    for study_fn in os.listdir(predictions_path):
        result = torch.load(os.path.join(predictions_path, study_fn))
        study_to_deltas[(result['patient'], result['study_id'])] = (result['labels'] - result['scores'], result['labels'], result['mask'], result['scores'])

    class_to_sorted_studies = {}
    for i, class_name in enumerate(classes):
        all_deltas = []
        for (patient_id, study_id), (deltas, labels, mask, scores) in study_to_deltas.items():
            all_deltas.append((patient_id, study_id, deltas[i], labels[i], mask[i], scores[i]))
        class_to_sorted_studies[class_name] = sorted(all_deltas, key=lambda x: x[2], reverse=True)
        
    return class_to_sorted_studies

In [57]:
frontal_class_to_sorted_studies = compute_class_to_sorted_studies(FRONTAL_PREDICTIONS_PATH, PAPER_TRAINING_CLASSES)
lateral_class_to_sorted_studies = compute_class_to_sorted_studies(LATERAL_PREDICTIONS_PATH, PAPER_TRAINING_CLASSES)

combined_class_to_sorted_studies = {}
for class_name in frontal_class_to_sorted_studies.keys():
    frontal = frontal_class_to_sorted_studies[class_name]
    lateral = lateral_class_to_sorted_studies[class_name]
    combined = [(f[0], f[1], f[2] - l[2], f[3], f[4], f[5], l[5]) for f, l in zip(frontal, lateral)]
    combined_class_to_sorted_studies[class_name] = sorted(combined, key=lambda x: x[2], reverse=True)

In [58]:
print(PAPER_TRAINING_CLASSES)
for i, class_name in enumerate(PAPER_TRAINING_CLASSES):
    count = 0
    studies = combined_class_to_sorted_studies[class_name]
    for study in studies:
        if count >= 10:
            break
        if study[3] == 1 and study[4] == 1:
            count += 1
            print('Class: {} Delta: {}, Path: {}, Label: {}, Mask: {}, Frontal Score: {} Lateral Score: {}'
                .format(class_name, study[2], os.path.join(DATASET_PATH, study[0], study[1]),
                        study[3], study[4], study[5], study[6]
            ))

['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']
Class: Cardiomegaly Delta: 0.0497046858072, Path: /home/kelvin.wong/Datasets/CheXpert-v1.0/train/patient45220/study1, Label: 1, Mask: 1.0, Frontal Score: 0.168845742941 Lateral Score: 0.218550428748
Class: Cardiomegaly Delta: 0.0492539703846, Path: /home/kelvin.wong/Datasets/CheXpert-v1.0/train/patient00111/study1, Label: 1, Mask: 1.0, Frontal Score: 0.162054926157 Lateral Score: 0.211308896542
Class: Cardiomegaly Delta: 0.042012989521, Path: /home/kelvin.wong/Datasets/CheXpert-v1.0/train/patient19818/study1, Label: 1, Mask: 1.0, Frontal Score: 0.357201218605 Lateral Score: 0.399214208126
Class: Cardiomegaly Delta: 0.0393134355545, Path: /home/kelvin.wong/Datasets/CheXpert-v1.0/train/patient10654/study2, Label: 1, Mask: 1.0, Frontal Score: 0.126344025135 Lateral Score: 0.16565746069
Class: Cardiomegaly Delta: 0.0385549739003, Path: /home/kelvin.wong/Datasets/CheXpert-v1.0/train/patient44580/study2, Label: 1, 